In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("valentynsichkar/traffic-signs-dataset-in-yolo-format")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/traffic-signs-dataset-in-yolo-format


In [2]:
!cat '/kaggle/input/traffic-signs-dataset-in-yolo-format/classes.names'

prohibitory
danger
mandatory
other


In [3]:
! pip install ultralytics opencv-python numpy matplotlib

In [8]:
import os
import shutil

# Define paths
data_dir = path + "/ts/ts"  # Change this to your actual dataset folder
train_dir = "/kaggle/working/dataset/train/"
val_dir = "/kaggle/working/dataset/val/"

# Create directories for images and labels
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Function to copy images and their corresponding label files
def copy_data(txt_file, dest_folder):
    with open(txt_file, "r") as f:
        filenames = [line.strip() for line in f.readlines()]  # Read image filenames

    for filename in filenames:
        justTheFileName = os.path.basename(filename)  # Extract filename (e.g., "IMAGE.jpg")
        justTheName, ext = os.path.splitext(justTheFileName)  # Split name and extension

        # Paths for image and label
        src_img = os.path.join(data_dir, justTheFileName)
        src_label = os.path.join(data_dir, justTheName + ".txt")  # Corresponding label

        dest_img = os.path.join(dest_folder, justTheFileName)
        dest_label = os.path.join(dest_folder, justTheName + ".txt")

        # Copy image if it exists
        if os.path.exists(src_img):
            shutil.copy(src_img, dest_img)
            print(f"Copied image: {src_img} -> {dest_img}")
        else:
            print(f"Missing image: {src_img}")

        # Copy label if it exists
        if os.path.exists(src_label):
            shutil.copy(src_label, dest_label)
            print(f"Copied label: {src_label} -> {dest_label}")
        else:
            print(f"Missing label: {src_label}")

# Copy images and labels to train and val folders
copy_data(path + "/train.txt", train_dir)
copy_data(path + "/test.txt", val_dir)

Copied image: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00433.jpg -> /kaggle/working/dataset/train/00433.jpg
Copied label: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00433.txt -> /kaggle/working/dataset/train/00433.txt
Copied image: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00570.jpg -> /kaggle/working/dataset/train/00570.jpg
Copied label: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00570.txt -> /kaggle/working/dataset/train/00570.txt
Copied image: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00358.jpg -> /kaggle/working/dataset/train/00358.jpg
Copied label: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00358.txt -> /kaggle/working/dataset/train/00358.txt
Copied image: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00091.jpg -> /kaggle/working/dataset/train/00091.jpg
Copied label: /kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts/00091.txt -> /kaggle/working/dataset/train/00091.txt
Copied i

In [9]:
import yaml
import os

class_names = ["prohibitory", "danger", "mandatory", "other"]
# Define dataset paths
data_config = {
    "train": train_dir,
    "val": val_dir,
    "nc": len(class_names),  # Number of classes. Fetched from the classes.names file
    "names": class_names # Replace with actual class names
}

# Save to data.yaml file
data_file_path = "/kaggle/working/data.yaml"
with open(data_file_path, "w") as file:
    yaml.dump(data_config, file, default_flow_style=False)

In [12]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8s.pt")

# Train model on your dataset
model.train(data=data_file_path, epochs=50, batch=16, imgsz=640)

Ultralytics 8.3.96 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

train: Scanning /kaggle/working/dataset/train.cache... 630 images, 0 backgrounds, 0 corrupt: 100%|██████████| 630/630 [00:00<?, ?it/s]

train: WARNING ⚠️ /kaggle/working/dataset/train/00340.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/dataset/val.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.08it/s]


                   all        111        179       0.93      0.918      0.965      0.836
           prohibitory         51         71       0.94      0.972      0.987      0.864
                danger         30         36      0.933      0.972      0.993      0.884
             mandatory         23         23      0.895       0.87       0.95      0.836
                 other         36         49      0.951      0.857      0.932      0.758


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aa0387eb640>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [13]:
model.val()

Ultralytics 8.3.96 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /kaggle/working/dataset/val.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  3.69it/s]


                   all        111        179       0.93      0.918      0.965      0.836
           prohibitory         51         71       0.94      0.972      0.987      0.873
                danger         30         36      0.933      0.972      0.993      0.886
             mandatory         23         23      0.896       0.87       0.95      0.829
                 other         36         49      0.951      0.857      0.932      0.756


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 1.8ms preprocess, 6.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train32


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aa168b42020>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0